# New methane lifetime treatment within FaIR

Use the option `RunMode.CH4LifetimeMethod = CH4LifetimeMethod.AERCHEMMIP`

We should try this first with concentrations and then with emissions, both with prescribed temperatures.

Then in another notebook, we will try a full emissions-driven run with freely evolving temperatures.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
from fair21.defaults import species_config_from_default

In [ ]:
samples = 25

In [ ]:
# note that this will not be correct for other SSPs

df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(751)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)
gmst[351:] = 4

In [ ]:
# TODO: change the SLCF_OZONE_PRECURSOR label to something more general; REACTIVE_GAS?

species_ids = {
    'CH4': SpeciesID('CH4', Category.CH4, run_mode=RunMode.EMISSIONS),
    'CO2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
    'N2O': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS, run_mode=RunMode.CONCENTRATION),
    'VOC': SpeciesID('VOC', Category.SLCF_OZONE_PRECURSOR),
    'NOx': SpeciesID('NOx', Category.SLCF_OZONE_PRECURSOR),
    'CO': SpeciesID('CO', Category.SLCF_OZONE_PRECURSOR),
}

In [ ]:
concentration_species = list(species_ids.keys())
emitted_species = ['CH4', 'VOC', 'NOx', 'CO']
for species in emitted_species:
    concentration_species.remove(species)
species_to_include = emitted_species + concentration_species
species_to_include

In [ ]:
scenarios_to_include = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
scenarios = []

for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2500'
        ].interpolate(axis=1).values.squeeze()
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
    
    for ispec, species in enumerate(concentration_species):
        species_rcmip_name = species.replace("-", "")
        conc_in = df_conc.loc[
            (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_conc['Region']=='World'), '1750':'2500'
        ].interpolate(axis=1).values.squeeze()
        list_of_species.append(Species(species_ids[species], concentration=conc_in))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
conc_ch4 = {}
for scenario in scenarios_to_include:
    conc_ch4[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()
conc_ch4

In [ ]:
list_of_species

In [ ]:
#ch4_lifetime = np.ones(samples) * scipy.stats.uniform.rvs(7.25, 2, size=samples, random_state=31018316)
#ch4_iirf_0 = np.ones(samples) * scipy.stats.uniform.rvs(7.25, 2, size=samples, random_state=10810610)
#ch4_iirf_temperature = np.ones(samples) * scipy.stats.uniform.rvs(-0.4, 0.35, size=samples, random_state=33372001)
#ch4_iirf_airborne = np.ones(samples) * scipy.stats.uniform.rvs(0.0001, 0.0005, size=samples, random_state=371210)
#ch4_baseline_concentration = scipy.stats.norm.rvs(729.2, 9.4, size=samples, random_state=121270)

ch4_lifetime = np.ones(samples) * 10.8537568 * 0.9776656032699657
ch4_soil_lifetime = np.ones(samples) * 185

In [ ]:
len(species_to_include)

In [ ]:
configs = []

seedgen = 1876963
for imodel in tqdm(range(samples)):
    config_name = f"{imodel}"
    climate_response = ClimateResponse()
    seedgen = seedgen+399
    species_config = [species_config_from_default(species) for species in species_to_include]
    # CH4 is index 0
    #species_config[0].iirf_0 = ch4_iirf_0[imodel]
    #species_config[0].iirf_airborne = ch4_iirf_airborne[imodel]
    #species_config[0].iirf_temperature = ch4_iirf_temperature[imodel]
    species_config[0].lifetime = ch4_lifetime[imodel]
    #species_config[0].baseline_concentration = ch4_baseline_concentration[imodel]
    species_config[0].natural_emissions_adjustment = 19.019783117809567
    species_config[0].soil_lifetime = ch4_soil_lifetime[imodel]
    #species_config[0].calculate_iirf0()  # needed if editing the other params
    #species_config[1].species_id = species_ids['co2']
    #species_config[2].species_id = species_ids['n2o']
    for isid, species_id in enumerate(species_to_include):
        species_config[isid].species_id = species_ids[species_id]
    
    configs.append(Config(config_name, climate_response, species_config))

In [ ]:
configs[1].species_configs[2]

In [ ]:
# default
RunConfig()

In [ ]:
# modify
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)
run_config

In [ ]:
start = time.time()
fair = FAIR(scenarios, configs, timestep=1, run_config=run_config)
fair.prescribe_temperature(gmst)
fair.run()
end = time.time()
print (f"{len(scenarios) * len(configs)} ensemble members in {end - start}s.")

In [ ]:
fair.concentration_array[:,0,0,0,0]

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16,9))
for i in range(8):
    ax[i//4, i%4].plot(fair.concentration_array[:,i,0,0,0]);
    ax[i//4, i%4].plot(conc_ch4[scenarios_to_include[i]][:], color='k')
    ax[i//4, i%4].set_title(scenarios_to_include[i])

In [ ]:
pl.plot(fair.emissions_array[:271,0,0,0,0]);

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0]);

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,:,0,0,0] * fair.lifetime_array[:,:,0,0,0]);

In [ ]:
pl.plot(fair.temperature[:,0,:,0,0]);

In [ ]:
pl.plot(fair.alpha_lifetime_array[:,0,:,0,0]);

In [ ]:
fair.alpha_lifetime_array.shape

In [ ]:
fair.lifetime_array.shape